По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

film id for calculations = 328

Выходной формат файла — json. Пример решения:

{
   "hist_film": [
      134,
      123,
      782,
      356,
      148
   ],
   "hist_all": [
      134,
      123,
      782,
      356,
      148
   ]
}
В поле “hist_film” нужно указать для заданного id фильма количество поставленных оценок в следующем порядке: "1", "2", "3", "4", "5". То есть, сколько было единичек, двоек, троек и т.д.

В поле “hist_all” нужно указать то же самое, только для всех фильмов общее количество поставленных оценок в том же порядке: "1", "2", "3", "4", "5"

In [66]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [67]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [73]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split('\t'))

In [74]:
rdd.take(5)
#  user id | item id | rating | timestamp

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

# rdd

In [76]:
hist_film = rdd.filter(lambda x: x[1]=='328')\
           .flatMap(lambda x: [x[2]])\
           .sortBy(lambda x: x[0])\
           .countByKey()
hist_all = df_all = rdd.flatMap(lambda x: [x[2]])\
           .sortBy(lambda x: x[0])\
           .countByKey()
hist_json = {"hist_film": list(df_302.values()),
             "hist_all": list(df_all.values())}

# dataframe

In [51]:
df = rdd.map(lambda x : (x[0], x[1], x[2], x[3])).toDF(("user_id","item_id","rating","timestamp"))

hist_film = df.filter(df['item_id'] == 328)\
              .groupBy([df['item_id'], df['rating']])\
              .count()\
              .sort(df['rating'], ascending=True)\
              .collect()
hist_all = df.groupBy(df['rating'])\
             .count()\
             .sort(df['rating'], ascending=True)\
             .collect()
hist_json = {"hist_film": [hist_film[i][2] for i in range(len(hist_film))],
             "hist_all": [hist_all[i][1] for i in range(len(hist_all))]}

In [ ]:
import json

with open("lab01.json", "w", encoding="utf-8") as file:
    json.dump(hist_json, file)

In [61]:
spark.stop()